### G27
Dave Brunner

In [75]:
import math

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt

In [76]:
if torch.cuda.is_available():
    print("cuda would be available, but not set")
    # torch.set_default_device('cuda')
else:
    print("cuda is not available")
if torch.backends.mps.is_available():
    print("mps would be available, but not set")
    mps_device = torch.device("mps")

cuda is not available
mps would be available, but not set


### Loading Data

In [77]:
training_data = datasets.mnist.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.mnist.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

In [78]:
training_data, validation_data = torch.utils.data.random_split(training_data, [50000, 10000])

In [79]:
print(len(training_data), len(validation_data), len(test_data))

50000 10000 10000


### Exercise 2: Fixed MLP with Increasing Training Dataset

Create a MLP with one hidden layer with 200 units for Fashion MNIST classification. Use ReLU activation.

Use a random fraction of the training set (split above) to perform the training. Always use the same validation set.

Use SGD and cross-entropy loss and suitable learning rate.

Start with a single small batch for training (batch size 8) and make sure that you can overfit, i.e. bring the training accuracy to 100%.

Then, gradually increase the training set. Let it grow until you obtain values for the training and the validation loss which no longer indicate overfitting. Use a fixed batch size (batchsize 32).

#### MLP Class

In [88]:
# define a method that provides an instance of an MLP which uses as a list of units per layer as input
# partial_training_data, rest = torch.utils.data.random_split(training_data, [2000, 48000])


def mlp(units=[28 * 28, 200, 10]):
    class MLP(torch.nn.Module):
        def __init__(self):
            super(MLP, self).__init__()
            self.sequential = torch.nn.Sequential(
                torch.nn.Flatten(),
                torch.nn.Linear(units[0], units[1]),
                torch.nn.ReLU(),
                torch.nn.Linear(units[1], units[2])
            )

        def forward(self, x):
            return self.sequential(x)

    return MLP()

In [89]:
# create an instance and its summary

model = mlp()
from torchsummary import summary

summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 200]         157,000
              ReLU-3                  [-1, 200]               0
            Linear-4                   [-1, 10]           2,010
Total params: 159,010
Trainable params: 159,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.61
Estimated Total Size (MB): 0.62
----------------------------------------------------------------


#### Training Loop 

In [91]:
def train_eval(model, lr, nepochs, nbatch, training_data, validation_data):
    cost_hist = []
    cost_hist_validation = []
    acc_hist = []
    acc_hist_validation = []

    cost_fn = torch.nn.CrossEntropyLoss()
    train_loader = DataLoader(training_data, batch_size=nbatch, shuffle=True)
    val_loader = DataLoader(validation_data, batch_size=nbatch, shuffle=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    iterations_pred_epoch = math.ceil(len(training_data) / nbatch)

    for epoch in range(nepochs + 1):
        acc_train = 0
        loss_train = 0
        for batch_idx, (train_x, train_y) in enumerate(train_loader):
            pred_train = model(train_x)
            loss = cost_fn(pred_train, train_y)

            # calculate performance scores
            acc_train += (pred_train.argmax(1).eq(train_y)).type(torch.float).sum().item()
            loss_train += loss.item()

            # update parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        cost_hist.append(loss_train / iterations_pred_epoch)
        acc_hist.append(acc_train / iterations_pred_epoch)

        with torch.no_grad():
            for val_x, val_y in enumerate(val_loader):
                pred = model(test_data)
                loss = cost_fn(pred, test_data)
                cost_hist_validation.append(loss.item())
                acc_val = (pred.argmax(1).eq(val_y)).type(torch.float).sum().item()
                acc_hist_validation.append(acc_val)
        print(f"Epoch: {epoch + 1} validation accuracy of {acc_val}")

    return cost_hist, cost_hist_validation, acc_hist, acc_hist_validation

#### First Training

Run a first training with only one small training batch (e.g. with a single batch of 64 samples). 
The small training set can be created by using the functionality `torch.utils.data.random_split` already used above. As validation set use the `validation_data` created above.  

This training run can be used to test whether the model and training loop are properly implemented. Explain why and in what sense it can be used as test.

This is something you can always do when training a model.

In [92]:
nbatch = 32
nbatches = 1
nepochs = 100
lr = 0.1

trainsize = nbatches * nbatch
trainset, rest = torch.utils.data.random_split(training_data, [trainsize, 50000 - trainsize])

print(trainset)
print(len(trainset), len(rest))

model = mlp([28 * 28, 200, 10])
cost_train, cost_valid, acc_train, acc_valid = train_eval(model, lr, nepochs, nbatch, trainset.dataset, validation_data.dataset)

plt.figure(1)
plt.plot(range(nepochs), cost_train, "b-")
plt.plot(range(nepochs), cost_valid, "r-")

plt.figure(2)
plt.plot(range(nepochs), acc_train, "b-")
plt.plot(range(nepochs), acc_valid, "r-")

32 49968


AttributeError: 'FashionMNIST' object has no attribute 'flatten'

#### Evaluate Train and Validation Performance 

Now run several trainings with the same small model (one hidden layer) and explore for different number of training samples (different number of batches with 32 samples) used, how the train and validation performance evolve (cost and accuracy). Make sure that you train sufficiently long to obtain representative values for cost and accuracy with the given settings. Always use the same validation set (with 10'000 samples).

Create plots with training and validation performance vs number of training batches (one for cost and one for accuracy). Use the performance characteristics obtained at the end.

Discuss the whether there is a sufficient number of training samples for the given problem at hand. Specify a minimum number if applicable. Also consider whether you are in the underfitting regime.

Hint: Specify a list with the number of training batches you would like to perform trainings. Try to be economic with the resources used - try to keep the number of trainings limited.

#### Comments: YOUR comments

### Exercise 3: Evaluate Different Model Complexities

Use the same functionality implemented above (create MLP model, train and evaluate model) to evaluate different model complexities: Number of layers and number of units per layer.

Start with the small model used in Exercise 2. Then gradually increase the model complexity. Do this along two dimensions:
* a single hidden layer, but increasing the number of units.
* a fixed number of units per (hidden) layer, but increase the number of layers.
Make sure that you reach the overfitting regime (in either case).

Always use the full training set with 50'000 samples.

Again make sure that you train sufficiently long so that the obtained train and validation performance measures (cost, accuracy) are representative.

Create plots with training and validation performance (cost, accuracy) vs model complexity - one plot with number of units for the single hidden layer case, and one for varying number of layers. 

Again use the performance characteristics obtained at the end. 

Finally, discuss your findings.

In [ ]:
# YOUR CODE HERE #
# several iterations with the code snippets of the form:

nbatch = 64
nepochs =
lr =

model =

cost_train, cost_valid, acc_train, acc_valid = train_eval(model, lr, nepochs, nbatch, training_data, validation_data)

plt.figure(1)
plt.plot(range(nepochs), cost_train, "b-")
plt.plot(range(nepochs), cost_valid, "r-")

plt.figure(2)
plt.plot(range(nepochs), acc_train, "b-")
plt.plot(range(nepochs), acc_valid, "r-")



#### Comments: YOUR findings.

### Exercise 4: Add Regularisation

Finally, add regularisation - dropout or L1/L2-regularisation. 

To this end, you need to implement new functionality to instantiate the model.

Start with one of the overfitting cases of Exercise 3 and try to improve the validation performance by adding regularisation. You can use either dropout or L1/L2-regularisation.


In [ ]:
# define a method that provides an instance of an MLP incl regularisation which uses as a list of units per layer as input 

def mlp_dropout(units=[28 * 28, 200, 10], p_in=0.2, p_hidden=0.5):
    """
    Creates an instance of an MLP with layers as specified in the 'units'-list (list of integers) and dropout 
    regularisation. Dropout rate for all layers the same except for the first (p_in). For the output layer 
    no dropout applied. 
    """

    # YOUR CODE HERE #



In [ ]:
model = mlp_dropout([28 * 28, 200, 10])

from torchsummary import summary

summary(model, (1, 28, 28))

#### Playing with different complexities and regularisation

Now play with different complexities and regularisation. 
Start with one of the overfitting cases identified in the previous exercise.
By adding regularisation, you should be able to make it non-overfitting, i.e. generalising better.
Note that for a given complexity, adding regularisation reduces the model capacity. This may need to be compensated by increasing the complexity of the model. 

Use again cost and accuracy for train and validation set to evaluate the results.

Finally, estimate the bias error and the generalisation error. 

In [ ]:

# YOUR CODE HERE #

